In [25]:
import sys
from operator import itemgetter
from random import randint
import random
import numpy as np
import time

alpha=0.1
beta=0.01
lmd=0.5
num_of_iterations=10
burnin=8
outputfile="output.txt"
inputfile_test="input-test.txt"
inputfile_train="input-train.txt"
K=10#the number of topics

In [26]:
data=None
with open(inputfile_train,'r') as f:
	data=f.readlines()
	f.close()
train_data=[]
for line in data:
	tokens=line.strip("\n").split(" ")
	train_data.append(tokens)


test_data1=None
with open(inputfile_test,'r') as f:
	test_data1=f.readlines()
	f.close()
test_data=[]
for line in test_data1:
	tokens=line.strip("\n").split(" ")
	test_data.append(tokens)

In [33]:
vocabulary_train=set()
vocabulary_test=set()
vocabulary=set()
for tokens in train_data:
    for token in tokens:
        vocabulary.add(token)
        vocabulary_train.add(token)
V_train=len(vocabulary)

for tokens in test_data:
    for token in tokens:
        vocabulary_test.add(token)
        vocabulary.add(token)
        
list1=list(vocabulary)
token2index={}
for i in range(len(list1)):
    token2index[list1[i]]=i

V_test=len(vocabulary_test)
V=len(vocabulary)

In [34]:
len(vocabulary)

8635

In [35]:
len(vocabulary_test)

5168

In [36]:
len(vocabulary_train)

8096

In [37]:
global_phi=[]
for i in range(V):
    global_phi.append([])
    for j in range(K):
        global_phi[i].append(0)
        


## create collection phi
collection_phi=[]
collection_phi.append([])
collection_phi.append([])


#collection_0_ph=[]
for i in range(V):
    collection_phi[0].append([])
    collection_phi[1].append([])
    for j in range(K):
        collection_phi[0][i].append(0)
        collection_phi[1][i].append(0)

doc_topics=[]

array=[]
x=[]
global_NumOftopics=[]
for i in range(K):
	global_NumOftopics.append(0)

c_NumOftopics=[]
c_NumOftopics.append([])
c_NumOftopics.append([])
for i in range(K):
	c_NumOftopics[0].append(0)
	c_NumOftopics[1].append(0)

In [38]:
for tokens in train_data:
    #tokens=line.strip("\n").split(" ")
    array.append([])
    doc_topics.append([])
    x.append([])
    size_of_doc=len(doc_topics)
    for i in range(K):
        doc_topics[size_of_doc-1].append(0)
    for token in tokens:
        #vocabulary.add(token)
        random_topic=randint(0,K-1)
        x_d_i=np.random.choice(2,1,p=[1-lmd,lmd])[0]
        x[len(x)-1].append(x_d_i)
        array[len(array)-1].append(random_topic)
        global_NumOftopics[random_topic]+=1
        # construct docs
        doc_topics[size_of_doc-1][random_topic]+=1
        global_phi[token2index[token]][random_topic]+=1
        
        c_NumOftopics[int(tokens[0])][random_topic]+=1
        collection_phi[int(tokens[0])][token2index[token]][random_topic]+=1
#             if tokens[0]=='1':
#             	c_NumOftopics[1][random_topic]+=1
#             	if token in collection_phi[1][random_topic]:
#             		collection_phi[1][random_topic][token]+=1
#             	else:
#             		collection_phi[1][random_topic][token]=1
#             elif tokens[0]=='0':
#             	c_NumOftopics[0][random_topic]+=1
#             	if token in collection_phi[0][random_topic]:
#             		collection_phi[0][random_topic][token]+=1
#             	else:
#             		collection_phi[0][random_topic][token]=1

In [39]:
global_NumOftopics

[12115, 12111, 12187, 12146, 12313, 12235, 12057, 12182, 12222, 12111]

In [40]:
x_test=[]
array_test=[]
doc_topics_test=[]
global_NumOftopics_test=[]
for i in range(K):
	global_NumOftopics_test.append(0)

c_NumOftopics_test=[]
c_NumOftopics_test.append([])
c_NumOftopics_test.append([])
for i in range(K):
	c_NumOftopics_test[0].append(0)
	c_NumOftopics_test[1].append(0)

In [41]:
d=-1
for tokens in test_data:
    d+=1
    #tokens=line.strip("\n").split(" ")
    array_test.append([])
    x_test.append([])
    doc_topics_test.append([])
    for i in range(K):
        doc_topics_test[len(doc_topics_test)-1].append(0)
        
    for i in range(len(tokens)):
        random_topic=randint(0,K-1)
        x_d_i=np.random.choice(2,1,p=[1-lmd,lmd])[0]

        global_NumOftopics_test[random_topic]+=1
        c_NumOftopics_test[int(tokens[0])][random_topic]+=1

        doc_topics_test[len(doc_topics_test)-1][random_topic]+=1
        x_test[len(x_test)-1].append(x_d_i)
        array_test[d].append(random_topic)

In [57]:
def get_distribution_doc_topic(flag,c_index,doc_index,token,doc_topics):
    global K
    global alpha
    global beta
    global vocabulary
    global global_phi
    global collection_phi
    
    distributions=[]
    if flag==1:
        for i in range(K):
            distribution_i=(doc_topics[doc_index][i]+alpha)/(len(array[doc_index])-1+K*alpha)
            sum_nk=c_NumOftopics[c_index][i]
            distribution_i*=(collection_phi[c_index][token2index[token]][i]+beta)/(sum_nk+V_train*beta)
#             if token in collection_phi[c_index][i]:
#             	distribution_i*=(collection_phi[c_index][i][token]+beta)/(sum_nk+V*beta)
#             else:
#             	distribution_i*=(beta)/(sum_nk+V*beta)
            distributions.append(distribution_i)
    elif flag==0:
        for i in range(K):
            distribution_i=(doc_topics[doc_index][i]+alpha)/(len(array[doc_index])-1+K*alpha)
            sum_nk=global_NumOftopics[i]
            distribution_i*=(global_phi[token2index[token]][i]+beta)/(sum_nk+V_train*beta)
#             if token in global_phi[i]:
#             	distribution_i*=(global_phi[i][token]+beta)/(sum_nk+V*beta)
#             else:
#             	distribution_i*=beta/(sum_nk+V*beta)
            distributions.append(distribution_i)
    
    return np.array(distributions)/np.sum(distributions)

In [61]:
def smaple_xdi(c_index,doc_index,token,x,z_d_token):
    global vocabulary
    global global_phi
    global collection_phi
    global lmd
    global V
    
    sum_nk=global_NumOftopics[z_d_token]
    p_0=(1-lmd)*((global_phi[token2index[token]][z_d_token]+beta)/(sum_nk+V_train*beta))
#     if token in global_phi[z_d_token]:
#     	p_0=(1-lmd)*((global_phi[z_d_token][token]+beta)/(sum_nk+V*beta))
#     else:
#     	p_0=(1-lmd)*(beta/(sum_nk+V*beta))

    sum_nk_c=c_NumOftopics[c_index][z_d_token]
    p_1=lmd*((collection_phi[c_index][token2index[token]][z_d_token]+beta)/(sum_nk_c+V_train*beta))
#     if token in collection_phi[c_index][z_d_token]:
#     	p_1=lmd*((collection_phi[c_index][z_d_token][token]+beta)/(sum_nk_c+V*beta))
#     else:
#     	p_1=lmd*(beta/(sum_nk_c+V*beta))
    
    dis= np.array([p_0 / (p_0+p_1), p_1 / (p_0+p_1)])#np.array(p)/np.sum(p)
    return np.random.choice(2,1,p=dis)[0]

In [59]:
def update_dic(z_d_token,token,token_index,doc_index,c_index,flag):
    global vocabulary
    global global_phi
    global collection_phi
    global doc_topics
    global array
    
    original_topic=array[doc_index][token_index]
    global_phi[token2index[token]][original_topic]-=1
    global_phi[token2index[token]][z_d_token]+=1
#     if token in global_phi[z_d_token]:
#     	global_phi[z_d_token][token]+=1
#     else:
#     	global_phi[z_d_token][token]=1
    collection_phi[c_index][token2index[token]][original_topic]-=1
    collection_phi[c_index][token2index[token]][z_d_token]+=1
#     if token in collection_phi[c_index][z_d_token]:
#     	collection_phi[c_index][z_d_token][token]+=1
#     else:
#     	collection_phi[c_index][z_d_token][token]=1
    array[doc_index][token_index]=z_d_token
    doc_topics[doc_index][original_topic]-=1
    doc_topics[doc_index][z_d_token]+=1
    
    global_NumOftopics[original_topic]-=1
    global_NumOftopics[z_d_token]+=1
    c_NumOftopics[c_index][original_topic]-=1
    c_NumOftopics[c_index][z_d_token]+=1  

In [ ]:
theta_info=[]
phi_c_info_array=[]
phi_g_info=[]


def generate_phi_theta():
	#theta_info=[]
	global theta_info
	global phi_c_info_array
	global phi_g_phi
	#theta_info=[]
	#phi_g_info=[]
	for i in range(K):
		phi_g_info.append({})
	
	#phi_c_info_array=[]
	phi_c_info_array.append([])
	phi_c_info_array.append([])
	for i in range(K):
		phi_c_info_array[0].append({})
		phi_c_info_array[1].append({})

	for i in range(len(doc_topics)):
		theta_info.append([])
		for j in range(K):
			temp=(doc_topics[i][j]+alpha)/(len(array[i])+K*alpha)
			theta_info[len(theta_info)-1].append(temp)
	
	for i in range(K):
		sum_nk=global_NumOftopics[i] 
		sum_nk_c1=c_NumOftopics[1][i]
		sum_nk_c0=c_NumOftopics[0][i]

		for ele in vocabulary:
			if ele in global_phi[i]:
				phi_g_info[i][ele]=(global_phi[i][ele]+beta)/(sum_nk+V*beta)
			else:
				phi_g_info[i][ele]=beta/(sum_nk+V*beta)
			
			if ele in collection_phi[0][i]:
				phi_c_info_array[0][i][ele]=(collection_phi[0][i][ele]+beta)/(sum_nk_c0+V*beta)
			else:
				phi_c_info_array[0][i][ele]=beta/(sum_nk_c0+V*beta)
			
			if ele in collection_phi[1][i]:
				phi_c_info_array[1][i][ele]=(collection_phi[1][i][ele]+beta)/(sum_nk_c1+V*beta)
			else:
				phi_c_info_array[1][i][ele]=beta/(sum_nk_c1+V*beta)

In [ ]:
-

iteration: 0
23.34278678894043
begin compute phi and theta
iteration: 1
